# 案例 1.5: 预测与早期预警

本案例演示了一个预测智能体如何根据历史数据进行预测，并发布预警。一个 `ARIMAForecaster` 智能体在接收历史数据后，会生成一个对未来的预测。如果预测值超过了预设的阈值，它将通过消息总线发布一个警告。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.local_agents.prediction.arima_forecaster import ARIMAForecaster

CONFIG_PATH = 'examples/forecasting_and_warning.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 设置消息总线和预警监听器

为了验证预警功能，我们设置一个简单的监听器函数，它会打印出接收到的任何预警消息。

In [ ]:
bus = MessageBus()
warning_topic = config['messaging']['warning_topic']
received_warnings = []

def warning_listener(message):
    print(f"\n>>> 预警接收成功! 主题: '{message['topic']}', 内容: {message['details']} <<<")
    received_warnings.append(message)

bus.subscribe(warning_topic, warning_listener)
print(f"已在主题 '{warning_topic}' 上设置预警监听器。")

### 3. 生成数据并运行预测

In [ ]:
def run_forecasting_example(cfg, message_bus):
    # 1. 创建预测智能体
    agent_cfg = cfg['agent_config']
    # 在此简化场景中，我们不使用真实的消息主题进行观测
    full_agent_cfg = {
        "observation_topic": "data/inflow", 
        "observation_key": "value",
        "forecast_topic": "forecast/inflow",
        **agent_cfg
    }
    agent = ARIMAForecaster(agent_id="InflowForecaster", message_bus=message_bus, config=full_agent_cfg)

    # 2. 生成历史数据
    data_cfg = cfg['data_generation']
    print("\n正在生成带有上升趋势的历史数据...")
    time_steps = np.arange(data_cfg['num_points'])
    history_data = pd.Series(
        np.sin(time_steps * data_cfg['sin_wave_multiplier']) * 10 + 
        (time_steps * data_cfg['trend_multiplier']) + 
        data_cfg['base_value']
    )
    agent.history.extend(history_data.tolist())

    # 3. 执行预测
    print(f"智能体已加载 {len(agent.history)} 个历史数据点，开始执行预测...")
    forecast = agent._fit_and_forecast() # 直接调用内部方法以简化示例

    if forecast:
        print(f"\n预测出未来 {len(forecast)} 个步长的值为: ")
        print([round(v, 2) for v in forecast])

        # 4. 检查是否需要发布预警
        warning_threshold = cfg['warning_threshold']
        print(f"\n正在检查预测值是否超过预警阈值 {warning_threshold}...")

        max_forecast_value = max(forecast)
        if max_forecast_value > warning_threshold:
            print(f"条件满足: 最大预测值 ({max_forecast_value:.2f}) > {warning_threshold}")
            warning_message = {
                "topic": message_bus.get_subscribed_topics()[0], # 获取监听器订阅的主题
                "details": f"预测到高流量. 最大预测值: {max_forecast_value:.2f} m^3/s."
            }
            message_bus.publish(warning_message['topic'], warning_message)
        else:
            print("条件未满足，未发布预警。")
    else:
        print("\n无法生成预测。")
        
    return history_data, forecast

history, forecast_values = run_forecasting_example(config, bus)

### 4. 可视化历史数据与预测结果

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(history.index, history, label='历史数据')
forecast_index = range(len(history), len(history) + len(forecast_values))
plt.plot(forecast_index, forecast_values, 'r--', label='预测值', marker='o')
plt.axhline(y=config['warning_threshold'], color='k', linestyle=':', label=f"预警阈值 ({config['warning_threshold']})")

plt.title('历史数据与ARIMA预测')
plt.xlabel('时间步')
plt.ylabel('数值')
plt.legend()
plt.grid(True)
plt.show()